<a href="https://colab.research.google.com/github/Abin-03/Nex_bot/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pandas numpy sentence-transformers scikit-learn transformers torch datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 33.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-many

In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 8.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration, Trainer, TrainingArguments
import torch
from datasets import Dataset
import gc

print("Libraries installed and imported.")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Libraries installed and imported.


In [ ]:
# Section 2: Load and Preprocess Data
def load_and_preprocess_data(file_path):
    df = pd.read_csv(file_path)
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df.columns = df.columns.str.strip()

    if 'Question' not in df.columns or 'Answer' not in df.columns:
        raise ValueError("CSV must contain 'Question' and 'Answer' columns")

    df['Question'] = df['Question'].apply(lambda x: x.lower().strip() if isinstance(x, str) else x)
    df['Answer'] = df['Answer'].apply(lambda x: x.strip() if isinstance(x, str) else x)
    df = df.dropna(subset=['Question', 'Answer'])

    return df['Question'].tolist(), df['Answer'].tolist()

# Replace 'your_dataset.csv' with the path to your CSV file
questions, answers = load_and_preprocess_data('/content/drive/MyDrive/ds.csv')
print(f"Loaded {len(questions)} question-answer pairs.")

Loaded 1649 question-answer pairs.


In [ ]:
print("Sample preprocessed data:")
print(questions[:5], answers[:5])

Sample preprocessed data:
['what is nexforce?', 'what is nexforce?', 'what services are offered by your company?', 'what are product creatives?', 'what are product creatives?'] ['NexForce is an initiative dedicated to transforming the educational landscape. NexForce is a program to help students learn through practical experience.', 'NexForce is a project started by students at Government Engineering College, Barton Hill, to change the way we learn. It helps students gain hands-on experience and learn by doing, making education more practical and useful.', 'We offer a wide range of services including Product Creatives, Web Development, Digital Marketing, Mobile App Development, Software Testing, Branding, ERP Software, E-Commerce Development, IoT, Embedded Systems, Social Media Marketing, SEO, Salesforce, API Integration, Graphic Designing, Media Production, 3D Design, UI/UX Design, Support and Consulting, Cyber Security, Machine Learning, MVP Development, Drones, Photography, and Expl

In [ ]:
# Section 3: Create Embeddings
def create_embeddings(questions):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    question_embeddings = model.encode(questions, show_progress_bar=True)
    gc.collect()
    return model, question_embeddings

embedding_model, question_embeddings = create_embeddings(questions)
print("Embeddings created.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/52 [00:00<?, ?it/s]

Embeddings created.


In [ ]:
# Section 4: Implement Retrieval System
def get_most_similar(query, top_k=3):
    query_embedding = embedding_model.encode([query])
    similarities = cosine_similarity(query_embedding, question_embeddings)
    top_indices = similarities.argsort()[0][-top_k:][::-1]
    return [(questions[i], answers[i], similarities[0][i]) for i in top_indices]

print("Retrieval system implemented.")


Retrieval system implemented.


In [ ]:
# Section 5: Set Up Language Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "facebook/blenderbot-400M-distill"
tokenizer = BlenderbotTokenizer.from_pretrained(model_name)
lm_model = BlenderbotForConditionalGeneration.from_pretrained(model_name).to(device)

def generate_response(context, max_length=150):
    inputs = tokenizer([context], return_tensors="pt", truncation=True, max_length=512).to(device)
    outputs = lm_model.generate(**inputs, max_length=max_length, num_return_sequences=1, do_sample=True, top_p=0.9, temperature=0.7)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Language model set up.")


tokenizer_config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/127k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/62.9k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/310k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/730M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

Language model set up.


In [ ]:
# Section 6: Create Chatbot Interface
def chatbot(query):
    query = query.lower().strip()
    retrieved_qa = get_most_similar(query)

    if retrieved_qa[0][2] > 0.8:
        return retrieved_qa[0][1]
    else:
        context = "Based on the following question-answer pairs, provide a relevant response:\n\n"
        for q, a, _ in retrieved_qa:
            context += f"Human: {q}\nAI: {a}\n\n"
        context += f"Human: {query}\nAI:"

        response = generate_response(context)
        return response.strip()

print("Chatbot interface created.")


Chatbot interface created.


In [ ]:
from datasets import Dataset
from transformers import Trainer, TrainingArguments

# Prepare data for fine-tuning
def prepare_data_for_training(questions, answers):
    data = {'input_text': questions, 'target_text': answers}
    dataset = Dataset.from_dict(data)

    def tokenize_function(examples):
        inputs = tokenizer(examples['input_text'], padding="max_length", truncation=True, max_length=128)
        targets = tokenizer(examples['target_text'], padding="max_length", truncation=True, max_length=128)
        return {
            'input_ids': inputs.input_ids,
            'attention_mask': inputs.attention_mask,
            'labels': targets.input_ids
        }

    tokenized_dataset = dataset.map(tokenize_function, batched=True)
    return tokenized_dataset

def fine_tune_model(questions, answers, epochs=3, batch_size=8):
    train_dataset = prepare_data_for_training(questions, answers)

    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=epochs,
        per_device_train_batch_size=batch_size,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
    )

    trainer = Trainer(
        model=lm_model,
        args=training_args,
        train_dataset=train_dataset,
    )

    trainer.train()

    # Save the fine-tuned model
    lm_model.save_pretrained("./fine_tuned_model")
    tokenizer.save_pretrained("./fine_tuned_model")

In [ ]:
import gc

# Run the chatbot
print("Chatbot: Hello! How can I assist you today? (Type 'quit' to exit, 'train' to fine-tune)")

while True:
    user_input = input("You: ")
    if user_input.lower() in ['quit', 'exit', 'bye']:
        print("Chatbot: Goodbye! Have a great day!")
        break
    elif user_input.lower() == 'train':
        print("Fine-tuning the model...")
        fine_tune_model(questions, answers)
        print("Fine-tuning complete!")
    else:
        response = chatbot(user_input)
        print("Chatbot:", response)

    # Clear some memory
    gc.collect()
    torch.cuda.empty_cache()


Chatbot: Hello! How can I assist you today? (Type 'quit' to exit, 'train' to fine-tune)
You: train
Fine-tuning the model...


Map:   0%|          | 0/1649 [00:00<?, ? examples/s]

Step,Training Loss
500,1.923000


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 60, 'min_length': 20, 'num_beams': 10, 'length_penalty': 0.65, 'no_repeat_ngram_size': 3, 'encoder_no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 60, 'min_length': 20, 'num_beams': 10, 'length_penalty': 0.65, 'no_repeat_ngram_size': 3, 'encoder_no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}


Fine-tuning complete!
You: what is nexforce
Chatbot: NexForce is a project started by students at Government Engineering College, Barton Hill, to change the way we learn. It helps students gain hands-on experience and learn by doing, making education more practical and useful.
You: why is ongoing skill development important for participants in NexForce
Chatbot: Ongoing skill development in NexForce ensures participants stay updated with industry trends, enhances their employability, and equips them with the necessary tools to tackle complex challenges in their chosen fields.
You: what is nexforce
Chatbot: NexForce is a project started by students at Government Engineering College, Barton Hill, to change the way we learn. It helps students gain hands-on experience and learn by doing, making education more practical and useful.
You: why is ongoing skill development important for participants in NexForce
Chatbot: Ongoing skill development in NexForce ensures participants stay updated with

In [ ]:
lm_model.save_pretrained("/content/drive/MyDrive/Nex_bot")
tokenizer.save_pretrained("/content/drive/MyDrive/Nex_bot")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 60, 'min_length': 20, 'num_beams': 10, 'length_penalty': 0.65, 'no_repeat_ngram_size': 3, 'encoder_no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}


('/content/drive/MyDrive/Nex_bot/tokenizer_config.json',
 '/content/drive/MyDrive/Nex_bot/special_tokens_map.json',
 '/content/drive/MyDrive/Nex_bot/vocab.json',
 '/content/drive/MyDrive/Nex_bot/merges.txt',
 '/content/drive/MyDrive/Nex_bot/added_tokens.json')

In [ ]:
print("Chatbot: Hello! How can I assist you today? (Type 'quit' to exit)")

while True:
    user_input = input("You: ")
    if user_input.lower() in ['quit', 'exit', 'bye']:
        print("Chatbot: Goodbye! Have a great day!")
        break
    else:
        response = chatbot(user_input)
        print("Chatbot:", response)


Chatbot: Hello! How can I assist you today? (Type 'quit' to exit)
You: What is nexforce


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
